<img src='https://github.com/LinkedEarth/Logos/raw/master/PYLEOCLIM_logo_HORZ-01.png' width="800">

# 4. Forcing and response

A common task in paleoclimatology is to relate a record (or several of them) to what is thought to be their forcing. A classic example comes $\delta^{18}O$ records from Chinese speleothems, broadly interpreted as reflecting continental-scale monsoon circulations, a frustratingly vague notion (see [Hu et al , 2019](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2019PA003741)).  In this notebook we focus on a composite of three Chinese speleothem records (Sanbao, Hulu, and Dongge caves) from [Cheng et al. (2016)](https://idp.nature.com/authorize/casa?redirect_uri=https://www.nature.com/articles/nature18591&casa_token=o9Vs_0ecaQMAAAAA:oo-GEcdU6XgZrRO9K08e-8uKxkeIx4RJciMSrOl5Qrur-i-C-trieshgEFve0npEBB-RWDzwhHzWJYG1) and its relationship to insolation.

As usual, we start by loading useful packages:

In [ ]:
import pandas as pd
import pyleoclim as pyleo
pyleo.set_style('web')

The composite wasn't available in LiPD form, so we do things by hand:

In [ ]:
lat_sb = 31.67 # latitude of the composite, defined
df = pd.read_csv('../data/Orbital_records/Sanbao_composite.csv')
tsb = pyleo.Series(time=df['age']/1000,time_name='Age',time_unit='ky BP',label='Sanbao/Hulu/Dongge x(-1)',
                  value=-df['d18O'],value_name=r'$\delta^{18}$O',value_unit=u'\u2030')
tsb.plot()

Note that we multiply the record by $-1$, as the record is conventionally interpreted as "more intense monsoon --> greater isotopic depletion". 

**Exercise 4.1** Use the `summary_plot` method to inspect the spectral content of this record. (to save time, you may set the number of significant tests to 0)

In [ ]:
## your work here

Sustained power in the ~20ky is evident, suggesting a link to orbital precession. Let us investigate this link in more detail:

## Constructing insolation curves

Speleothem records from Asia are often compared to summertime insolation (though the latitude of this field is not always consistently chosen). To compute the insolation, we make use of the excellent package [climlab](https://climlab.readthedocs.io/en/latest/index.html) by Brian Rose.  Lucky you, the package is already installed on this JupyterHub, so all we have to do is invoke it.

In [ ]:
from climlab import constants as const
from climlab.solar.orbital import OrbitalTable
from climlab.solar.insolation import daily_insolation
import numpy as np

kyears = np.linspace(-400, 0, 1001 ) # specify time interval and units
orb = OrbitalTable.interp(kyear=kyears) # subset of orbital parameters for specified time
days = np.linspace(0, const.days_per_year, 365)

Qsb = daily_insolation(lat_sb, days, orb) # generate insolation  at Sanbao latitude
Qsb_jja = np.mean(Qsb[:,151:243], axis=1)  # Julian days 152-243 are JJA

Note that calendar effects are non-negligible at those scales [(Bartlein & Shafer, 2019)](https://gmd.copernicus.org/articles/12/3889/2019/). To be perfectly rigorous, we should take those into account. But this is a hackathon, so let's proceed.

## Orbital Coherence

Given the scalogram  above, the real question is: what sort of phase lag is there between summertime insolation and the isotopic record? To answer this, let's put the heating into a _Series_ object so we can unlock some `pyleoclim` magic.

In [ ]:
qsb = pyleo.Series(time=-kyears,time_name='Age',time_unit='ky BP',label='JJA insolation',
                   value=Qsb_jja,value_name='Insolation',value_unit=r'$W.m^{-2}$' )
qsb.plot()

**Exercise 4.2** Plot the two on the same graph. Because of the different units, you might want to standardize first.

In [ ]:
## your code here##

There does seem to be extremely coherent behavior at orbital scales, something that we can confirm using
using wavelet coherence:

In [ ]:
coh = qsb.wavelet_coherence(tsb)
coh.plot()

**Exercise 4.3** Interpret this plot, leaning on the documentation of `coherence`. What do arrows pointing to the right mean? What coherence levels are significant, and with respect to what null hypothesis?

We do see fairly incoherent behavior at short time scales. That is to be expected, as the insolation curve is a very smooth, modulated harmonic (or nearly so) coming from theoretical calculations, while the speleothem composite is a real-world dataset with real-world uncertainties. Before spending valuable CPU time to compute significance limits (a non-trivial endeavor), let's break out some possible tools to clean this up. 

### Clean-up tool #1: Lowpass filter.

Given that all the noise appears to be at scales shorter than ~5ky, the simplest thing is to filter them out. Of course, `pyleoclim` has a button for that.

**Exercise 4.4** Look up the `filter()` function for the _Series_ class and apply with a cutoff scale of 10 kyr. Then plot it on top of the original, with proper labels.

**Answer 4.4**

In [ ]:
## your work here

### Clean-up tool #2: Singular spectrum analysis (SSA)

You've played a bit with SSA in notebook 3, where it did not get a full chance to shine: those series had extremely irregular sampling, which made life rather difficult for it. Let's see what SSA makes of this one. However, this turns out to be quite a long series for SSA, and the decomposition takes an impossibly long while. Since our purpose is to isolate long wavelengths (low frequencies), it's just as convenient to first coarse-grain the series. For this, we use `gkernel`, which is quite good at smoothing along the way:

In [ ]:
tsb_coarse = tsb.gkernel(step=2)
fig, ax = tsb.plot(mute=True,label='original')
tsb_coarse.plot(ax=ax,label='gkernel')
pyleo.showfig(fig)

Let's try SSA reconstruction on this new series:

**Exercise 4.5** Experiment with SSA truncation methods to find a tradeoff that best isolates features of interest.

In [ ]:
## your code here ##

**Exercise 4.6** Compare the two filtering methods, apply a summary plot to them

In [ ]:
## your code here ##

Now we can plot the filtered series next to the insolation curve. 

**Exercise 4.7** Put the two series on a common timescale (hint: use [common_time()](https://pyleoclim-util.readthedocs.io/en/latest/core/ui.html#pyleoclim.core.ui.MultipleSeries.common_time)), call them `qs` and `ts`, and plot them together.

In [ ]:
## your code here ##

**Exercise 4.8** Recompute wavelet coherence with the coarse-grained, filtered isotopic record. What changes? Was it worth it? How would you summarize the relationship between insolation and the Sanbao/Hulu/Dongge composite?

In [ ]:
## your code here ##

Now, this plot is still messy at high frequencies, and because they don't mean anything here, let's truncate them:

In [ ]:
coh.plot(ylim=[10,200])

## Causality

It's been known for about 300 years that correlation does not imply causation, so several methods have been devised to try to extract the notion of _causality_ from timeseries data.  `Pyleoclim` offers access to two of them:
- [Granger causality](http://www.scholarpedia.org/article/Granger_causality)
- [Liang-Kleeman information flow](http://www.ncoads.org/article/show/28.aspx)

**(optional) Exercise 4.9** Run the two and compare their output. How do you interpret it?

In [ ]:
caus = qs.causality(ts, settings={'nsim': 2000, 'signif_test': 'isopersist'})
print(caus)

In [ ]:
caus_granger = qs.causality(ts, method = 'granger')
print(caus_granger)